## Debug the SparsDT
import lib

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
import torch
import numpy as np
import lib.models as model

In [4]:
Y = torch.rand([10,1600],dtype=torch.float32)

### Testing estimating alpha

In [5]:
SparsDT = model.ModelSparseDT(10,Y, num_samples=1e5)

================= estimating alpha =================


### Testing the optimizer

In [7]:
args = {
    'lr' : 0.1
}
SparsDT.set_optimizer('SGD', args)

### Testing  estim gamma

In [12]:
U = torch.randn([SparsDT.m_rows, 1000])
a = SparsDT._estimate_gamma(U)
a.shape

torch.Size([1000])

### Testing find U, norm alpha, Loss

In [18]:
U = SparsDT._findU(1000)
U.shape

torch.Size([10, 1000])

In [21]:
norm = SparsDT._norm_alpha(SparsDT.A, U)
norm.shape

torch.Size([1000])

In [25]:
gamma = SparsDT._estimate_gamma(U)
l = SparsDT.Loss(U,gamma)
l

tensor(0.1757, grad_fn=<MeanBackward1>)

### Testing fit

In [31]:
A = SparsDT.fit(max_iter=1e3, rand_U=True, num_col = 100)

In [33]:
A

tensor([[0.0998, 0.1480, 0.3857, 0.2743, 0.2340, 0.4496, 0.4147, 0.4628, 0.6434,
         0.2958],
        [0.0764, 0.2042, 0.2962, 0.0567, 0.1072, 0.3108, 0.3265, 0.1237, 0.1079,
         0.2641],
        [0.4468, 0.3363, 0.3066, 0.4584, 0.4586, 0.3840, 0.4274, 0.3066, 0.1375,
         0.2081],
        [0.3095, 0.0185, 0.3475, 0.2308, 0.3421, 0.3220, 0.4224, 0.3423, 0.2060,
         0.4105],
        [0.0667, 0.4627, 0.2048, 0.4704, 0.4450, 0.3485, 0.1790, 0.0597, 0.2667,
         0.3652],
        [0.4613, 0.1825, 0.2587, 0.0406, 0.3714, 0.3689, 0.1250, 0.1582, 0.0673,
         0.2081],
        [0.2698, 0.4357, 0.3731, 0.3369, 0.0953, 0.2913, 0.3691, 0.4004, 0.1355,
         0.0479],
        [0.0789, 0.3275, 0.3908, 0.2798, 0.2864, 0.1856, 0.3102, 0.3814, 0.1725,
         0.1016],
        [0.4986, 0.3990, 0.3370, 0.1338, 0.3875, 0.0140, 0.0512, 0.4039, 0.5778,
         0.4055],
        [0.3312, 0.3095, 0.1226, 0.4444, 0.0549, 0.2263, 0.2260, 0.1728, 0.0933,
         0.5040]])

In [70]:
def estim_alpha(num_samples, batch=2000):
    """
    Estimating the alpha using the algorithm in
    "Reconstruction of Ultrasound RF Echoes Modeled as Stable Random Variables"
    num_samples : int
        number of samples in the estimator 
    batch : int
        number of added samples in each iteration
    returns alpha_hat
    """
    m = Y.shape[0]
    alphahat = 0
    count = 0
    while count <= num_samples:
        UT = torch.randn([batch,m])
        sig = (1e-7 + UT.mm(Y).abs()).log()           # (batch X k)
        print(sig.min(), UT.mm(Y).abs().min())
        k1 = sig.mean(dim=1).unsqueeze(1)    # (batch X 1)
        k2 = ((sig-k1)**2).mean(dim=1)       # (batch X 1)
        alpha_sq = (2 * np.pi**2 /(12*k2*(1 - np.pi**2/(12*k2))))
        #if 0<alpha_sq<4 :
        alpha_sq[alpha_sq<0] = 0
        alpha = alpha_sq ** 0.5
        alphahat += alpha.sum() 
        count += batch
    alpha = alphahat/count
    return alpha

In [72]:
Y = torch.randn([5, 16000], device= 'cpu')

In [309]:
Y = torch.randn([10, 1600],dtype=torch.float32)

In [164]:
optimizer = torch.optim.SGD

In [167]:
optimizer2 = type(optimizer)[A] 

TypeError: 'type' object is not subscriptable

In [204]:
idx = (0.5>A) * (A<2)

In [205]:
A[idx]

tensor([0.2644, 0.3029, 0.1076, 0.4197], grad_fn=<IndexBackward>)

In [179]:
AT = A.transpose(1,0)

In [186]:
AT.shape

torch.Size([16, 4])

In [187]:
AT = AT / AT.data.norm(dim=1).unsqueeze(dim=1)

In [223]:
U = []

In [268]:
A = torch.rand([5,16],dtype=torch.float32,requires_grad=True)
C = torch.tensor([])
B = torch.cat([A,A],dim=1)

In [21]:
x = None
b = 4 if x else 10
b

10

In [259]:
C = torch.tensor([])

In [264]:
AT.transpose(1,0).shape

torch.Size([4, 16])